# DB Scan Clustering Implementation

### Importing Dependencies

In [71]:
import numpy as np
import pickle
import sys
import time
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
from sklearn.decomposition import PCA as sklearnPCA
init_notebook_mode(connected=True)  #Set jupyter notebook mode to true for running pyplot

### Preprocess the input data

**Input Parameters** : file name


**returns**: output matrix X, disease id and the ground truth

In [72]:
def preprocess(filename):
    inpdata = np.genfromtxt(filename,delimiter = '\t')
    X = np.loadtxt(filename,delimiter = '\t', usecols = range(2, inpdata.shape[1]), dtype = 'S15')
    gen_id = np.loadtxt(filename,delimiter = '\t', usecols = 0, dtype = 'S15')
    ground_truth = np.loadtxt(filename,delimiter = '\t', usecols = 1, dtype = 'S15')
    return X, gen_id, ground_truth

### Run principle component analysis to convert n-dimensional data to 2 dimensions in order to visualize
**Input Parameters** : input matrix X


**returns**: Eigen Vector Y

In [73]:
def runPCA(X):
    sklearn_pca = sklearnPCA(n_components=2)
    Y_sklearn = sklearn_pca.fit_transform(X)
    return Y_sklearn

## Function to run the DB Scan Clustering Algorithm

**Input Parameters** : input data, epsilon, minPts, distance matrix, visited nodes, Cluster memory.


**returns**: clusters

In [74]:
def runDBSCAN(data, eps, minPts, distance_matrix, visited, memCluster):
    clusters = []
    noise = []
    visited = [False]*len(data)
    memCluster = [False]*len(data)
    for i in range(0, len(data)):
        if(visited[i] == False):
            visited[i] = True
            nPoints = regionQuery(i, eps, distance_matrix)
            if(len(nPoints) < minPts):
                noise.append(data[i])
            else:
                cluster = []
                cluster = expandCluster(data, i, nPoints, cluster, eps, minPts, distance_matrix, visited, memCluster)
                clusters.append(cluster)
    return clusters

### Function to expand the cluster
**Input Parameters** : input data, pointIndex, no of points, cluster, epsilon, minPts, distance matrix, visited nodes, Cluster memory.


**returns**: cluster

In [82]:
def expandCluster(data, pointIndex, nPoints, cluster, eps, minPts, distance_matrix, visited, memCluster):
    cluster.append(data[pointIndex])
    memCluster[pointIndex] = True
    for i in nPoints:
        if(visited[i] == False):
            visited[i] = True
            nPoints2 = regionQuery(i, eps, distance_matrix)
            if(len(nPoints2) >= minPts):
                nPoints += nPoints2
        if(memCluster[i] == False):
            cluster.append(data[i])
            memCluster[i] = True
    return cluster

In [83]:
def regionQuery(pointIndex, eps, distance_matrix):
    points = []
    for j in range(0, len(distance_matrix)):
        if(distance_matrix[pointIndex][j] <= eps):
            points.append(j)
    return points

### Draw Scatter Plot using plotly

**Input Parameters** : 2 dimentional data and its label


**prints**: visualized clusters

In [77]:
def draw_scatter_plot(Y, labels):
    unique_labels = set(labels)
    points = []
    for name in unique_labels:
        x = []
        y = []
        for i in range(0, len(labels)):
            if(labels[i] == name):
                x.append(Y[i,0])
                y.append(Y[i,1])
        x = np.array(x)
        y = np.array(y)
        point = Scatter(
            x = x,
            y = y,
            mode='markers',
            name = int(name),
            marker=Marker(size=12, line=Line(color='rgba(217, 154, 217, 123)',width=0.5),opacity=0.9))
        points.append(point)
    data = Data(points)
    layout = Layout(xaxis=XAxis(title='Principle Component 1', showline=True),
                    yaxis=YAxis(title='Principle Component 2', showline=True))
    fig = Figure(data=data, layout=layout)
    iplot(fig)

### Calculate the Jaccard value of predicted data

**Input Parameters** : actual ground truth matrix and predicted ground truth


**returns**: None


**prints**: Jaccard and Rand value on console

In [78]:
def calculateJackard(actual_ground_truth, predicted_ground_truth):
    m00, m01, m10, m11 = 0, 0, 0, 0
    for i in range(0, len(actual_ground_truth)):
        for j in range(0, len(actual_ground_truth)):
            if((actual_ground_truth[i] != actual_ground_truth[j]) and (predicted_ground_truth[i] != predicted_ground_truth[j])):
                m00 += 1
            elif((actual_ground_truth[i] == actual_ground_truth[j]) and (predicted_ground_truth[i] != predicted_ground_truth[j])):
                m01 += 1
            elif((actual_ground_truth[i] != actual_ground_truth[j]) and (predicted_ground_truth[i] == predicted_ground_truth[j])):
                m10 += 1
            elif((actual_ground_truth[i] == actual_ground_truth[j]) and (predicted_ground_truth[i] == predicted_ground_truth[j])):
                m11 += 1
    jaccard = m11 / float(m11 + m10 + m01)
    rand = (m11 + m00) / float(m11 + m10 + m01 + m00)
    print(" Jaccard is : " + str(jaccard)),
    print(" Rand is : " + str(rand))

## Driver program to run the above code

**Input Parameters** : file name, iteration count and initial centroids


**prints**: Jackard value and scatter plot

In [79]:
def driver(file_name, min_points, eps):
    db = DBSCAN(eps=1.03, min_samples=4, metric="precomputed")
    X, gen_id, ground_truth = preprocess(file_name)
    len_X = X.shape[0]
    distance_matrix = np.zeros((len_X, len_X), dtype='float64')
    print(X.shape)
    for i in range(0, len_X):
        for j in range(0, len_X):
            if(i != j):
                dis = 0
                for k in range(0, X.shape[1]):
                    dis = dis + np.square(np.subtract(float(X[i][k]), float(X[j][k])))
                distance_matrix[i][j] = np.sqrt(dis) 
            else:
                distance_matrix[i][j] = 0
    start = time.time()
    clusters = runDBSCAN(gen_id, eps, min_points, distance_matrix, [], [])
    print("Time to run is : "),
    print("--- %s seconds ---" % (time.time() - start))
    density_ground_truth = [-1]*len(gen_id)
    print("Clusters no :" + str(len(clusters)))
    for i in range(0, len(clusters)):
        for j in clusters[i]:
            density_ground_truth[int(j)-1] = i
    calculateJackard(ground_truth, density_ground_truth)
    Y_pca = runPCA(X)
    draw_scatter_plot(Y_pca, density_ground_truth)

## To run this algorithm for a different file or no of clusters, please change parameters here

In [80]:
FILENAME = "data/cho.txt"
EPS = 1.03
MIN_POINTS = 4
driver(FILENAME, MIN_POINTS, EPS)

(386, 16)
Time to run is : 
--- 0.0612330436706543 seconds ---
Clusters no :5
 Jaccard is : 0.20318706260639305
 Rand is : 0.5476120164299713


In [81]:
FILENAME = "data/new_dataset_1.txt"
EPS = 1.03
MIN_POINTS = 4
driver(FILENAME, MIN_POINTS, EPS)

(150, 4)
Time to run is : 
--- 0.011939048767089844 seconds ---
Clusters no :2
 Jaccard is : 0.6
 Rand is : 0.7777777777777778
